## Model

In [1]:
import transformers

print(transformers.__version__)

4.39.0


In [2]:
import pandas as pd
import numpy as np
import torch
import random
from torch.utils.data import Dataset, DataLoader
from transformers import set_seed, AutoTokenizer, AutoModel, DistilBertTokenizer, DistilBertForSequenceClassification

from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [3]:
import datasets
import random
from IPython.display import display, HTML

In [4]:
#!pip install datasets

In [5]:
torch.cuda.is_available()

True

## Helper functions

In [6]:
class AbstractsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: self.encodings[key][idx] for key in self.encodings}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Note the dtype change for binary classification
        return item

    def __len__(self):
        return len(self.labels)

## Set seed

In [7]:
set_seed(6013)

## DistillBERT

In [8]:
dat = pd.read_csv('zotero_data.csv')
dat['abstract'] = dat['abstract'].astype(str)

In [9]:
labels = dat['fus_related'].tolist()
print("Label distribution:", np.bincount(labels))

Label distribution: [1794 1794]


In [10]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenized_data = tokenizer(list(dat['abstract']), padding=True, truncation=True, max_length=512, return_tensors="pt")

In [11]:
dataset = AbstractsDataset(tokenized_data, labels)

In [12]:
# Split indices into train and test sets first
train_idx, test_idx = train_test_split(list(range(len(dataset))), test_size=0.1, random_state=42)

# Split train indices into train and validation sets
train_idx, val_idx = train_test_split(train_idx, test_size=0.1, random_state=42)

# Create Subset datasets
train_dataset = torch.utils.data.Subset(dataset, train_idx)
val_dataset = torch.utils.data.Subset(dataset, val_idx)
test_dataset = torch.utils.data.Subset(dataset, test_idx)
print(f"Dataset size - train: {len(train_dataset)}, val: {len(val_dataset)}, test: {len(test_dataset)}")

Dataset size - train: 2906, val: 323, test: 359


In [13]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=0.001, 
    report_to=None,
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    logging_strategy="epoch",  # Log at the end of each epoch
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', zero_division=1)  # Adjust zero_division here
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

## Hyper-param tuning

In [14]:
#! pip install ray[tune]

In [15]:
import ray
from ray import train, tune

In [16]:
ray.init()

2024-04-10 15:53:07,144	INFO worker.py:1752 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.10.0


In [17]:
# Print the status of the Ray cluster.
print('''This cluster consists of
    {} nodes in total
    {} CPU resources in total
    {} Memory resources in total
    {} Object store memory resources in total
    '''.format(len(ray.nodes()), ray.cluster_resources()['CPU'], 
           ray.cluster_resources()['memory'] / (1024*1024*1024), 
           ray.cluster_resources()['object_store_memory'] / (1024*1024*1024)))

This cluster consists of
    1 nodes in total
    48.0 CPU resources in total
    151.474424934946 Memory resources in total
    68.90899314824492 Object store memory resources in total
    


In [18]:
def model_init():
    return DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

In [19]:
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.create_optimizer_and_scheduler(num_training_steps=1000)
trainer.optimizer = torch.optim.AdamW(trainer.model.parameters(),lr=5e-5)
trainer.lr_scheduler = get_linear_schedule_with_warmup(trainer.optimizer, num_warmup_steps=500, num_training_steps=1000)

/home/re4mfy/.local/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream ta

In [20]:
def ray_hp_space(trial):
    return {
        "per_device_train_batch_size": tune.grid_search([8, 16, 32]),
        "learning_rate": tune.grid_search([2e-5, 3e-5, 5e-5]),
        "num_train_epochs": tune.grid_search([2, 3, 4])
    }

In [21]:
best_run = trainer.hyperparameter_search(n_trials=1, 
                                        direction="maximize",
                                        hp_space=ray_hp_space)

2024-04-10 15:53:36,699	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-04-10 15:53:37 (running for 00:00:00.35)
Using FIFO scheduling algorithm.
Logical resource usage: 0/48 CPUs, 0/4 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2024-04-10_15-53-03_117292_14815/artifacts/2024-04-10_15-53-36/_objective_2024-04-10_15-53-36/driver_artifacts
Number of trials: 27/27 (27 PENDING)
+------------------------+----------+-------+-----------------+--------------------+------------------------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_bat |
|                        |          |       |                 |                    |                ch_size |
|------------------------+----------+-------+-----------------+--------------------+------------------------|
| _objective_05023_00000 | PENDING  |       |           2e-05 |                  2 |                      8 |
| _objective_05023_00001 | PENDING  |       |           3e-05 |                  2 |     

(_objective pid=18285) /home/re4mfy/.local/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
(_objective pid=18285) dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
(_objective pid=18285)   warnings.warn(
(_objective pid=18285) Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
(_objective pid=18285) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifie

== Status ==
Current time: 2024-04-10 15:54:48 (running for 00:01:11.19)
Using FIFO scheduling algorithm.
Logical resource usage: 4.0/48 CPUs, 4.0/4 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2024-04-10_15-53-03_117292_14815/artifacts/2024-04-10_15-53-36/_objective_2024-04-10_15-53-36/driver_artifacts
Number of trials: 27/27 (23 PENDING, 4 RUNNING)
+------------------------+----------+--------------------+-----------------+--------------------+------------------------+
| Trial name             | status   | loc                |   learning_rate |   num_train_epochs |   per_device_train_bat |
|                        |          |                    |                 |                    |                ch_size |
|------------------------+----------+--------------------+-----------------+--------------------+------------------------|
| _objective_05023_00000 | RUNNING  | 10.155.48.78:18282 |           2e-05 |                  2 |                      8 |
| _object

  0%|          | 1/1092 [00:03<1:07:49,  3.73s/it]
(_objective pid=18282) /home/re4mfy/.local/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead:  [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_objective pid=18282) dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True) [repeated 3x across cluster]
(_objective pid=18282)   warnings.warn( [repeated 3x across cluster]
(_objective pid=18282) Detected kernel version 4.18.0, which is below the recommended m

...

== Status ==
Current time: 2024-04-10 16:20:14 (running for 00:26:37.90)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/48 CPUs, 1.0/4 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2024-04-10_15-53-03_117292_14815/artifacts/2024-04-10_15-53-36/_objective_2024-04-10_15-53-36/driver_artifacts
Number of trials: 27/27 (1 RUNNING, 26 TERMINATED)
+------------------------+------------+--------------------+-----------------+--------------------+------------------------+-------------+
| Trial name             | status     | loc                |   learning_rate |   num_train_epochs |   per_device_train_bat |   objective |
|                        |            |                    |                 |                    |                ch_size |             |
|------------------------+------------+--------------------+-----------------+--------------------+------------------------+-------------|
| _objective_05023_00026 | RUNNING    | 10.155.48.78:23758 |     

(_objective pid=23758) 
(_objective pid=23758) 
(_objective pid=23758) 
                                                 A
100%|██████████| 364/364 [01:43<00:00,  3.52it/s]A


(_objective pid=23758) {'eval_loss': 0.19262978434562683, 'eval_accuracy': 0.9535603715170279, 'eval_f1': 0.9560117302052786, 'eval_precision': 0.9421965317919075, 'eval_recall': 0.9702380952380952, 'eval_runtime': 0.8978, 'eval_samples_per_second': 359.756, 'eval_steps_per_second': 23.39, 'epoch': 4.0}
(_objective pid=23758) {'train_runtime': 103.4968, 'train_samples_per_second': 112.313, 'train_steps_per_second': 3.517, 'train_loss': 0.3294027139852335, 'epoch': 4.0}


2024-04-10 16:20:15,784	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/home/re4mfy/ray_results/_objective_2024-04-10_15-53-36' in 0.3529s.
2024-04-10 16:20:15,798	INFO tune.py:1048 -- Total run time: 1599.10 seconds (1598.37 seconds for the tuning loop).


== Status ==
Current time: 2024-04-10 16:20:15 (running for 00:26:38.73)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/48 CPUs, 1.0/4 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2024-04-10_15-53-03_117292_14815/artifacts/2024-04-10_15-53-36/_objective_2024-04-10_15-53-36/driver_artifacts
Number of trials: 27/27 (27 TERMINATED)
+------------------------+------------+--------------------+-----------------+--------------------+------------------------+-------------+
| Trial name             | status     | loc                |   learning_rate |   num_train_epochs |   per_device_train_bat |   objective |
|                        |            |                    |                 |                    |                ch_size |             |
|------------------------+------------+--------------------+-----------------+--------------------+------------------------+-------------|
| _objective_05023_00000 | TERMINATED | 10.155.48.78:18282 |           2e-05

In [22]:
for n, v in best_run.hyperparameters.items():
    print(n, v)

per_device_train_batch_size 16
learning_rate 3e-05
num_train_epochs 3


In [23]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.504000,0.208958,0.931889,0.935673,0.919540,0.952381
2,0.241500,0.250574,0.922601,0.924471,0.938650,0.910714
3,0.213800,0.153579,0.953560,0.957020,0.922652,0.994048


TrainOutput(global_step=546, training_loss=0.3197894812503577, metrics={'train_runtime': 75.7658, 'train_samples_per_second': 115.065, 'train_steps_per_second': 7.206, 'total_flos': 1154850781483008.0, 'train_loss': 0.3197894812503577, 'epoch': 3.0})

In [25]:
trainer.evaluate(eval_dataset=test_dataset)

{'eval_loss': 0.2885777950286865,
 'eval_accuracy': 0.9025069637883009,
 'eval_f1': 0.9066666666666666,
 'eval_precision': 0.8374384236453202,
 'eval_recall': 0.9883720930232558,
 'eval_runtime': 0.9337,
 'eval_samples_per_second': 384.474,
 'eval_steps_per_second': 24.632,
 'epoch': 3.0}

In [26]:
trainer.predict(test_dataset=test_dataset)

PredictionOutput(predictions=array([[-1.1651036 ,  0.85192525],
       [-1.202273  ,  0.99878454],
       [-1.2997152 ,  1.0840393 ],
       [ 3.4333186 , -3.2757795 ],
       [-1.26693   ,  1.0261086 ],
       [ 3.2827365 , -3.1301858 ],
       [ 3.3502803 , -3.2794795 ],
       [ 3.420917  , -3.345804  ],
       [ 3.4200544 , -3.3165488 ],
       [-1.2986714 ,  1.0854062 ],
       [ 2.7971003 , -2.8669696 ],
       [-1.2457974 ,  1.0112944 ],
       [ 3.3098803 , -3.2609468 ],
       [-1.247654  ,  1.0046122 ],
       [ 3.4837751 , -3.3215594 ],
       [-1.1723346 ,  0.84335124],
       [ 3.4614289 , -3.3440225 ],
       [-1.2220099 ,  0.9472104 ],
       [ 3.419617  , -3.2689502 ],
       [ 3.3422422 , -3.226693  ],
       [ 3.4817748 , -3.312643  ],
       [-1.2300178 ,  1.0127162 ],
       [-1.2477082 ,  1.0014645 ],
       [-1.2668068 ,  1.0066255 ],
       [-1.2305408 ,  0.969898  ],
       [-1.1300038 ,  0.87620246],
       [-1.2753515 ,  1.081683  ],
       [ 3.4173698 , -3.30